This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Reference frames

## Sample-detector geometry

The figure below shows the [sample reference frame](#detector-sample-geometry)
and the [detector reference frame](#detector-coordinates) used in kikuchipy, all
of which are right handed. In short, the sample reference frame is the one used
by EDAX TSL, RD-TD-ND, while the pattern center is defined as in the Bruker
software.

<img id="detector-sample-geometry" src="../_static/image/reference_frames/detector_sample_geometry.png">

In **(a)** (lower left), a schematic of the microscope chamber shows the
definition of the sample reference frame, RD-TD-ND. The
$x_{euler}-y_{euler}-z_{euler}$ crystal reference frame used by Bruker is shown
for reference. An EBSD pattern on the detector screen is viewed from *behind*
the screen *towards* the sample. The inset **(b)** shows the detector and sample
normals viewed from above, and the azimuthal angle $\omega$ which is defined as
the sample tilt angle round the RD axis. **(c)** shows how the EBSD map appears
within the data collection software, with the sample reference frame and the
scanning reference frame, $x_{scan}-y_{scan}-z_{scan}$, attached. Note the
$180^{\circ}$ rotation of the map about ND. **(d)** shows the relationship
between the sample reference frame and the detector reference frame,
$x_{detector}-y_{detector}-z_{detector}$, with the projection center
highlighted. The detector tilt $\theta$ and sample tilt $\sigma$, in this case
$10^{\circ}$ and $70^{\circ}$, respectively, are also shown.

<img id="detector-coordinates" src="../_static/image/reference_frames/detector_gnomonic_coordinates.jpg">

The above figure shows the EBSD pattern in the
[sample reference frame figure](#detector-sample-geometry) (a) as viewed from
behind the screen towards the sample (left), with the detector reference frame
the same as in (d) with its origin (0, 0) in the upper left pixel. The detector
pixels' gnomonic coordinates can be described with a calibrated projection
center (PC) (right), with the gnomonic reference frame origin (0, 0) in ($PC_x,
PC_y$). The circles indicate the angular distance from the PC in steps of
$10^{\circ}$.

## The EBSD detector

All relevant parameters for the sample-detector geometry are stored in an
[kikuchipy.detectors.EBSDDetector](../reference/generated/kikuchipy.detectors.EBSDDetector.rst)
instance. Let's first import necessary libraries and a small Nickel EBSD test
data set

In [ ]:
# Exchange inline for notebook or qt5 (from pyqt) for interactive plotting
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp


s = kp.data.nickel_ebsd_small()  # Use kp.load("data.h5") to load your own data
s

Then we can define a detector with the same parameters as the one used to
acquire the small Nickel data set

In [ ]:
detector = kp.detectors.EBSDDetector(
    shape=s.axes_manager.signal_shape[::-1],
    pc=[0.421, 0.779, 0.505],
    convention="tsl",
    px_size=70,  # microns
    binning=8,
    tilt=0,
    sample_tilt=70,
)
detector

In [ ]:
detector.pc_tsl()

The projection/pattern center (PC) is stored internally in the Bruker
convention:

- $PC_x$ is measured from the left border of the detector in fractions of detector
  width.

- $PC_y$ is measured from the top border of the detector in fractions of detector
  height.

- $PC_z$ is the distance from the detector scintillator to the sample divided by
  pattern height.

Above, the PC was passed in the EDAX TSL convention. Passing the PC in the
Bruker, Oxford, or EMsoft v4 or v5 convention is also supported. The definitions
of the conventions are given in the
[EBSDDetector](../reference/generated/kikuchipy.detectors.EBSDDetector.rst)
reference, together with the conversion from PC coordinates in the TSL, Oxford,
or EMsoft conventions to PC coordinates in the Bruker convention.

The PC coordinates in the TSL, Oxford, or EMsoft conventions can be retreived
via [EBSDDetector.pc_tsl()](../reference/generated/kikuchipy.detectors.EBSDDetector.pc_tsl.rst),
[EBSDDetector.pc_oxford()](../reference/generated/kikuchipy.detectors.EBSDDetector.pc_oxford.rst),
and [EBSDDetector.pc_emsoft()](../reference/generated/kikuchipy.detectors.EBSDDetector.pc_emsoft.rst),
respectively. The latter requires the unbinned detector pixel size in microns
and the detector binning to be given upon initialization.

In [ ]:
detector.pc_emsoft()

The detector can be plotted to show whether the average PC is placed as
expected using
[EBSDDetector.plot()](../reference/generated/kikuchipy.detectors.EBSDDetector.plot.rst)
(see its docstring for a complete explanation of its parameters)

In [ ]:
detector.plot(pattern=s.inav[0, 0].data)

This will produce a figure similar to the left panel in the
[detector coordinates figure](#detector-coordinates) above, without the arrows
and colored labels.

Multiple PCs with a 1D or 2D navigation shape can be passed to the `pc`
parameter upon initialization, or can be set directly. This gives the detector
a navigation shape (not to be confused with the detector shape) and a navigation
dimension (maximum of two)

In [ ]:
detector.pc = np.ones([3, 4, 3]) * [0.421, 0.779, 0.505]
detector.navigation_shape

In [ ]:
detector.navigation_dimension

In [ ]:
detector.pc = detector.pc[0, 0]
detector.navigation_shape

<div class="alert alert-info">

Note

The offset and scale of HyperSpy’s `axes_manager` is fixed for a signal,
meaning that we cannot let the PC vary with scan position if we want to
calibrate the EBSD detector via the `axes_manager`. The need for a varying
PC was the main motivation behind the `EBSDDetector` class.
    
</div>

The right panel in the [detector coordinates figure](#detector-coordinates)
above shows the detector plotted in the gnomonic projection using
[EBSDDetector.plot()](../reference/generated/kikuchipy.detectors.EBSDDetector.plot.rst).
We assign 2D gnomonic coordinates ($x_g, y_g$) in a gnomonic projection plane
parallel to the detector screen to a 3D point ($x_d, y_d, z_d$) in the detector
frame as

$$
x_g = \frac{x_d}{z_d}, \qquad y_g = \frac{y_d}{z_d}.
$$

The detector bounds and pixel scale in this projection, per navigation point,
are stored with the detector

In [ ]:
detector.bounds

In [ ]:
detector.gnomonic_bounds

In [ ]:
detector.x_range

In [ ]:
detector.r_max  # Largest radial distance to PC